# Analysis of cooperating algorithms 
### (not finalized)

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [7]:
filenames = ["data/run" + str(i) + "/loop-1_composite_objf_vs_eval_proxy_train_yx.dat" for i in range(1, 6)]

In [76]:
filenames

['data/run1/loop-1_composite_objf_vs_eval_proxy_train_yx.dat',
 'data/run2/loop-1_composite_objf_vs_eval_proxy_train_yx.dat',
 'data/run3/loop-1_composite_objf_vs_eval_proxy_train_yx.dat',
 'data/run4/loop-1_composite_objf_vs_eval_proxy_train_yx.dat',
 'data/run5/loop-1_composite_objf_vs_eval_proxy_train_yx.dat']

In [77]:
dfs_orig = [pd.read_csv(filename) for filename in filenames]

In [78]:
def add_run_id(df, id):    
    df['RUNID'] = id          
    columns = df.columns.tolist()
    return df[columns[-1:] + columns[:-1]]

def add_run_ids(dfs):
    for i, df in enumerate(dfs):
        print(i)
        dfs[i] = add_run_id(df, i+1)
    return dfs

In [79]:
# add the run'ids
dfs = add_run_ids(dfs_orig)

0
1
2
3
4


In [80]:
# concatenate the dataframes
dfs = pd.concat(dfs)

In [81]:
dfs.describe()

,RUNID,OBJF,X1,X2,X3,X4,X5,X6,X7,X8,...,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48
count,57000.000000,5.700000e+04,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,...,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000
mean,2.927895,5.505619e+08,0.432282,0.654839,0.417031,0.437807,0.544396,0.461076,0.427563,0.542297,...,0.526859,0.508407,0.510892,0.458244,0.468358,0.547518,0.505104,0.393349,0.504737,0.441092
std,1.318528,2.735622e+09,0.262234,0.265096,0.227516,0.228031,0.228545,0.229425,0.246271,0.214252,...,0.289923,0.215712,0.215544,0.182973,0.252569,0.317089,0.265378,0.237598,0.222941,0.230324
min,1.000000,-2.445000e+10,0.000542,0.000052,0.000348,0.000064,0.000411,0.000480,0.000500,0.000090,...,0.000111,0.000249,0.000089,0.000204,0.000326,0.000029,0.000145,0.000212,0.000186,0.000066
25%,2.000000,-2.920289e+08,0.229674,0.477256,0.224890,0.239438,0.314107,0.281199,0.246153,0.450125,...,0.254547,0.359405,0.346731,0.342684,0.274927,0.274678,0.324259,0.154060,0.306823,0.217916
50%,3.000000,-2.504767e+08,0.332800,0.687913,0.325443,0.451499,0.558872,0.464320,0.449064,0.544764,...,0.525757,0.594172,0.497281,0.463477,0.481930,0.594949,0.460752,0.319950,0.514389,0.407892
75%,4.000000,-1.657659e+08,0.590260,0.888705,0.566884,0.589461,0.727487,0.656969,0.620212,0.649287,...,0.738058,0.629895,0.710913,0.584576,0.719179,0.825899,0.748031,0.565829,0.705882,0.596928
max,5.000000,1.000000e+10,0.999911,0.999512,0.999506,0.999586,0.999879,0.999565,0.999920,0.999839,...,0.999945,0.999910,0.999674,0.997211,0.998322,0.998655,0.999913,0.999612,0.999888,0.999688


In [86]:
df1 = dfs[dfs.RUNID==1]

In [87]:
df1

,RUNID,OBJF,X1,X2,X3,X4,X5,X6,X7,X8,...,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48
0,1,-8.501940e+07,0.332800,0.039200,0.305900,0.661700,0.946500,0.848600,0.267500,0.540500,...,0.378800,0.877300,0.922800,0.667000,0.453200,0.537700,0.848500,0.334500,0.752500,0.724000
1,1,-1.430508e+08,0.332800,0.039200,0.305900,0.661700,0.946500,0.848600,0.267500,0.540500,...,0.659889,0.052445,0.107059,0.584993,0.758935,0.172929,0.078109,0.057551,0.356724,0.779963
2,1,-2.140565e+08,0.332800,0.039200,0.305900,0.661700,0.946500,0.848600,0.267500,0.540500,...,0.232399,0.361612,0.277985,0.049247,0.664464,0.264273,0.669980,0.777562,0.504592,0.491920
3,1,-1.807461e+08,0.065541,0.027816,0.152155,0.333449,0.070106,0.208748,0.626339,0.433911,...,0.378800,0.877300,0.922800,0.667000,0.453200,0.537700,0.848500,0.334500,0.752500,0.724000
4,1,-7.503419e+07,0.924193,0.390793,0.662870,0.727935,0.465852,0.043654,0.300683,0.492993,...,0.378800,0.877300,0.922800,0.667000,0.453200,0.537700,0.848500,0.334500,0.752500,0.724000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9925,1,-2.518528e+08,0.469574,0.598363,0.700509,0.405189,0.415218,0.736647,0.482433,0.495667,...,0.138462,0.699329,0.836555,0.643677,0.719179,0.925848,0.352587,0.421914,0.185997,0.143542
9926,1,1.000000e+10,0.614913,0.599139,0.524158,0.410429,0.468654,0.562105,0.787389,0.383049,...,0.396884,0.612285,0.058551,0.682434,0.545007,0.628448,0.848684,0.732760,0.750613,0.747963
9927,1,-2.873171e+08,0.614913,0.599139,0.524158,0.410429,0.468654,0.562105,0.787389,0.383049,...,0.338451,0.843530,0.350929,0.638178,0.739094,0.295605,0.309047,0.315294,0.230848,0.384050
9928,1,-1.928092e+08,0.614913,0.599139,0.524158,0.410429,0.468654,0.562105,0.787389,0.383049,...,0.366230,0.895419,0.557826,0.026380,0.172308,0.585042,0.488261,0.317220,0.763822,0.399433


In [ ]:
# Fill in the line below: get names of columns with missing values
columns_with_missing_values = [col for col in X_train.columns if X_train[col].isnull().any()] # Your code here

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(columns_with_missing_values, axis=1)
reduced_X_valid = X_valid.drop(columns_with_missing_values, axis=1)

In [ ]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer=SimpleImputer() # Your code here
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [ ]:
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

In [ ]:
# drop categorical variables
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

In [ ]:
# OrdinalEncoding
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

In [ ]:
# OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

In [ ]:
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

In [ ]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

In [ ]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)